<a href="https://colab.research.google.com/github/mashruravi/learning-pytorch/blob/master/01_pytorch_cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN for MNIST hand-written digit classification using Pytorch

In [3]:
import sys
sys.version

'3.6.9 (default, Nov  7 2019, 10:44:02) \n[GCC 8.3.0]'

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision as tv
import torchvision.datasets as ds

print("torch version:", torch.__version__)
print("torchvision version:", tv.__version__)

torch version: 1.4.0
torchvision version: 0.5.0


## Download MNIST dataset

In [5]:
# Transforms to normalize images
transform = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0,), (1,))
])

# Download dataset
train_dataset = ds.MNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

test_dataset = ds.MNIST(
    root='./data',
    train=False,
    transform=transform,
    download=True
)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


## Create Data Loaders

In [0]:
BS=32

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BS,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BS,
    shuffle=False
)

## Create Model

In [0]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
    self.bn1 = nn.BatchNorm2d(8)
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(16)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
    self.bn3 = nn.BatchNorm2d(32)
    self.classifier = nn.Linear(32 * 3 * 3, 10)
    self.pool = nn.MaxPool2d(2)

  def forward(self, x):
    x = F.dropout2d(self.pool(F.relu(self.bn1(self.conv1(x)))), p=0.05)
    x = F.dropout2d(self.pool(F.relu(self.bn2(self.conv2(x)))), p=0.05)
    x = F.dropout2d(self.pool(F.relu(self.bn3(self.conv3(x)))), p=0.05)
    x = x.view(-1, 32*3*3)
    x = self.classifier(x)
    return x

model = Net()

## Check for GPU availability

In [76]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model.to(device)

cuda:0


Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (classifier): Linear(in_features=288, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [77]:
from torchsummary import summary

summary(model, input_size=(1, 28, 28), batch_size=BS)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [32, 8, 28, 28]              80
       BatchNorm2d-2            [32, 8, 28, 28]              16
         MaxPool2d-3            [32, 8, 14, 14]               0
            Conv2d-4           [32, 16, 14, 14]           1,168
       BatchNorm2d-5           [32, 16, 14, 14]              32
         MaxPool2d-6             [32, 16, 7, 7]               0
            Conv2d-7             [32, 32, 7, 7]           4,640
       BatchNorm2d-8             [32, 32, 7, 7]              64
         MaxPool2d-9             [32, 32, 3, 3]               0
           Linear-10                   [32, 10]           2,890
Total params: 8,890
Trainable params: 8,890
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.10
Forward/backward pass size (MB): 6.01
Params size (MB): 0.03
Estimated Total

## Define a loss function and an optimizer

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Train the network

In [79]:
from time import time

EPOCHS=5
LOG_FREQ=100

# Set model in training mode
model.train()

for epoch in range(EPOCHS):

    print(f'Epoch {epoch + 1}')
    running_loss = 0
    start_time = time()

    # Loop over each batch
    for i, (inputs, labels) in enumerate(train_loader):

        # Send to GPU if available
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        running_loss += loss.item()

        if (i+1) % LOG_FREQ == 0:
          print(f'[{epoch+1}, {i+1}] loss: {(running_loss/LOG_FREQ):.3f}')
          running_loss = 0

    end_time = time()
    print(f'{end_time - start_time:.2f}s for epoch')

Epoch 1
[1, 100] loss: 1.544
[1, 200] loss: 0.622
[1, 300] loss: 0.415
[1, 400] loss: 0.312
[1, 500] loss: 0.276
[1, 600] loss: 0.238
[1, 700] loss: 0.205
[1, 800] loss: 0.207
[1, 900] loss: 0.167
[1, 1000] loss: 0.156
[1, 1100] loss: 0.157
[1, 1200] loss: 0.154
[1, 1300] loss: 0.141
[1, 1400] loss: 0.147
[1, 1500] loss: 0.135
[1, 1600] loss: 0.135
[1, 1700] loss: 0.140
[1, 1800] loss: 0.112
13.68s for epoch
Epoch 2
[2, 100] loss: 0.124
[2, 200] loss: 0.104
[2, 300] loss: 0.106
[2, 400] loss: 0.110
[2, 500] loss: 0.115
[2, 600] loss: 0.089
[2, 700] loss: 0.108
[2, 800] loss: 0.119
[2, 900] loss: 0.103
[2, 1000] loss: 0.118
[2, 1100] loss: 0.097
[2, 1200] loss: 0.091
[2, 1300] loss: 0.102
[2, 1400] loss: 0.099
[2, 1500] loss: 0.097
[2, 1600] loss: 0.091
[2, 1700] loss: 0.085
[2, 1800] loss: 0.085
13.42s for epoch
Epoch 3
[3, 100] loss: 0.095
[3, 200] loss: 0.077
[3, 300] loss: 0.080
[3, 400] loss: 0.070
[3, 500] loss: 0.089
[3, 600] loss: 0.081
[3, 700] loss: 0.072
[3, 800] loss: 0.080


## Check performance on test data

In [80]:

correct = 0
total = 0

model.eval()

for i, (inputs, labels) in enumerate(test_loader):

    # Send to GPU if available
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Test accuracy: {100 * correct / total}')

Test accuracy: 98.17
